In [1]:
import numpy as numpy
import pandas as pd

In [2]:
dataframe = pd.read_csv("dataset/psc_severity_test.csv")
dataframe

,PscInspectionId,deficiency_code,def_text,InspectionDate,VesselId,PscAuthorityId,PortId,VesselGroup,age
0,1802364,14402,PscInspectionId: 1802364\n\nDeficiency/Finding...,2024-04-05,293691,9,936,Dry Bulk,9.593429
1,1736765,10199,PscInspectionId: 1736765\n\nDeficiency/Finding...,2023-08-17,272075,9,5237,Dry Bulk,25.210130
2,1787907,18204,PscInspectionId: 1787907\n\nDeficiency/Finding...,2024-02-15,302667,1,953,Dry Bulk,5.793292
3,1691176,14108,PscInspectionId: 1691176\n\nDeficiency/Finding...,2023-03-13,288591,7,1439,Oil,12.446270
4,1712454,5109,PscInspectionId: 1712454\n\nDeficiency/Finding...,2023-05-26,290457,2,1366,Dry Bulk,11.731691
...,...,...,...,...,...,...,...,...,...
1096,1721780,1139,PscInspectionId: 1721780\n\nDeficiency/Finding...,2023-06-26,277654,7,1797,Chemical,18.732375
1097,1674478,14604,PscInspectionId: 1674478\n\nDeficiency/Finding...,2023-01-11,303135,9,3152,Dry Bulk,2.529774
1098,1778352,18203,PscInspectionId: 1778352\n\nDeficiency/Finding...,2024-01-10,293467,7,1459,General Cargo,14.275154
1099,1643351,10112,PscInspectionId: 1643351\n\nDeficiency/Finding...,2022-09-15,293373,9,2135,Dry Bulk,10.176591
